In [1]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/train-en-ms.tar.gz
# !tar -zxf train-en-ms.tar.gz
# !rm train-en-ms.tar.gz

In [2]:
# !pip3 install pyenchant

In [2]:
# import enchant
# d = enchant.Dict("en_US")
# d.check("Hello")

In [3]:
# !wget https://raw.githubusercontent.com/huseinzol05/malay-dataset/master/normalization/en-lexicon/en-social-media-lexicon.json
# !wget https://raw.githubusercontent.com/huseinzol05/malay-dataset/master/normalization/en-lexicon/en-social-media-lexicon-v2.json

In [4]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [5]:
!ls -lha train-en

total 1.6G
drwxr-xr-x  2 ubuntu ubuntu 4.0K Jun  28  2020 .
drwxr-xr-x 43 ubuntu ubuntu  12K Jul  12 22:03 ..
-rw-r--r--  1 ubuntu ubuntu 772M Ogos  2  2020 left.txt
-rw-r--r--  1 ubuntu ubuntu 860M Ogos  2  2020 right.txt


In [6]:
with open('train-en/left.txt') as fopen:
    left = fopen.read().split('\n')
    
with open('train-en/right.txt') as fopen:
    right = fopen.read().split('\n')

In [7]:
left[7106], right[7106]

('The goal is to make sure the people who receive the land are the ones who really deserve it.',
 'Tujuannya bagi memastikan pihak yang menerima tanah itu adalah mereka yang benar-benar layak.')

In [8]:
len(left), len(right)

(3807616, 3807616)

In [9]:
import malaya

/home/ubuntu/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.5.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(
/home/ubuntu/.local/lib/python3.8/site-packages/tensorflow_addons/utils/resource_loader.py:72: UserWarning: You are currently using TensorFlow 2.6.0 and trying to load a custom op (custom_ops/seq2seq/_beam_search_ops.so).
TensorFlow Addons has compiled its custom ops again

In [10]:
from malaya.text.rules import rules_normalizer, rules_compound_normalizer
from malaya.text.normalization import _is_number_regex
from collections import defaultdict
import re
import random
import json

In [11]:
with open('en-social-media-lexicon.json') as fopen:
    en_lexicon = json.load(fopen)
    
with open('en-social-media-lexicon-v2.json') as fopen:
    en_lexicon_v2 = json.load(fopen)

In [29]:
compound_normalizer = defaultdict(list)
normalizer = defaultdict(list)

for k, v in en_lexicon.items():
    if not len(v):
        continue
    if len(k.split()) > 1:
        compound_normalizer[k].extend(v)
    else:
        normalizer[k].extend(v)

for k, v in en_lexicon_v2.items():
    if not len(v):
        continue
    if len(k.split()) > 1:
        compound_normalizer[k].extend(v)
    else:
        normalizer[k].extend(v)
        
compound_normalizer_regex = (
    '(?:' + '|'.join(list(compound_normalizer.keys())) + ')'
)

In [41]:
PUNCTUATION = '!"#$%&\'()*+,./:;<=>?@[\]^_`{|}~'

def case_of(text):
    return (
        str.upper
        if text.isupper()
        else str.lower
        if text.islower()
        else str.title
        if text.istitle()
        else str
    )

def strip_punct(word):
    left = []
    right = []
    i = 0
    while i < len(word):
        if word[i] in PUNCTUATION:
            left.append(word[i])
            i += 1
        else:
            break
    i = len(word) - 1
    while i > 0:
        if word[i] in PUNCTUATION:
            right.append(word[i])
            i -= 1
        else:
            break
    left = ''.join(left)
    right = ''.join(right[::-1])
    if len(right):
        word_ = word[:-len(right)]
    else:
        word_ = word
    word_ = word_[len(left):]
    return left, right, word_


def replace_shortword(word, rules = normalizer):
    left, right, word_ = strip_punct(word)
    word_ = word_[len(left):]
    lower_word_ = word_.lower()
    if lower_word_ in rules:
        word_ = case_of(word_)(random.choice(rules[lower_word_]))
        word_ = f'{left}{word_}{right}'
        return word_
    else:
        return word
    
strip_punct('counters,')

('', ',', 'counters')

In [25]:
# def _replace_compound(string, 
#                       rules_regex = rules_compound_normalizer_regex, 
#                       rules = rev_rules_compound_normalizer):
#     results = re.findall(rules_regex, string, flags=re.IGNORECASE
#     )
#     for r in results:
#         try:
#             string = string.replace(r, random.choice(rules[r.lower()]))
#         except:
#             pass
#     return string

def _replace_compound(string, rules = compound_normalizer):
    for k in list(rules.keys()):
        results = [(m.start(0), m.end(0)) for m in re.finditer(k, string, flags=re.IGNORECASE)]
        for r in results:
            sub = string[r[0]: r[1]]
            try:
                replaced = random.choice(rules[sub.lower()])
                if replaced:
                    if r[1] < len(string) and string[r[1]] != ' ':
                        continue
                    if r[0] - 1 > len(string) and string[r[0] - 1] != ' ':
                        continue

                    sub = case_of(sub)(replaced)
                    string = string[:r[0]] + sub + string[r[1]:]
            except:
                pass
    return string

In [46]:
_replace_compound('i like, text to cell', rules = compound_normalizer)

'i like, ttc'

In [33]:
string = 'i am so beautiful"'
[replace_shortword(word) for word in string.split()]

['iiii', 'amt', 'soooooo', 'bueityful"']

In [42]:
[replace_shortword(word) for word in left[1].split()]

['Pitt',
 'haves',
 'snc',
 'addopted*2',
 'buth',
 'clraner.',
 'Iin',
 '2006,',
 'thge',
 'cupple',
 'head',
 "its'",
 'fast',
 'biological',
 'dauter,',
 'Shiloh',
 'Nouvel,',
 'hole',
 'prefers',
 'tew',
 'by*3',
 'cald',
 'Johhn,',
 'acording*2',
 '2',
 'They*6',
 'Telagraph.',
 'Thge',
 'cupple',
 'addopted*2',
 'Pax',
 'Thien,',
 'hor',
 '12,',
 'pefor',
 'Jolie',
 'cave*6',
 'berth',
 'tp',
 'twins',
 'Knox',
 'Leon',
 'anbd',
 'Vivienne',
 'Marcheline,',
 'noe',
 'seven.',
 'I',
 'Januari,',
 'Jolie',
 'maked',
 'headlines',
 'of*3',
 'bringin',
 'hier',
 'kidz',
 'ko',
 'thaa',
 'premeire',
 'on*2',
 '"Kung',
 'Fu',
 'Panda',
 '3".',
 'Jolie',
 'andd',
 'Pitt',
 'alls',
 'reportedly',
 'bareng',
 'aling',
 'and',
 'teem',
 'with',
 'nannies',
 'than',
 'tichers',
 'wereever',
 'their',
 'travill.',
 'Acording*2',
 'ko',
 'Jolie,',
 'millionnaire',
 'moms',
 'shouldnt',
 'complane',
 'to',
 'juggling',
 'workk',
 'i',
 'ffam',
 'becau',
 'thay',
 'haven',
 'moar',
 'acess,',
 

In [44]:
def replace_words_punct(left_word, right_word):
    left_left, left_right, left_word = strip_punct(left_word)
    right_left, right_right, right_word = strip_punct(right_word)
    return f'{left_left}{right_word}{left_right}'

def random_replace_alignment(left, right, alignment, min_replace = 2, max_replace = 7):
    splitted_left = left.split()
    splitted_right = right.split()
    
    selected_alignment = []
    for s in alignment:
        l = s[0]
        r = s[1]
        if _is_number_regex(splitted_left[l].replace(',', '').replace('.', '')) or _is_number_regex(splitted_right[r].replace(',', '').replace('.', '')):
            continue
        elif splitted_left[l].isupper() or splitted_right[r].isupper():
            continue
        elif splitted_left[l] == splitted_right[r]:
            continue
        elif splitted_right[r].lower() in ['the', 'a', 'an', 'it', 'is', 'are']:
            continue
        else:
            selected_alignment.append((l, r))
    
    count_replace = random.randint(min_replace, min(max_replace, len(selected_alignment)))
    
    selected = random.sample(selected_alignment, count_replace)
    for s in selected:
        splitted_left[s[0]] = replace_words_punct(splitted_left[s[0]], splitted_right[s[1]])
    
    return ' '.join(splitted_left)

In [45]:
# random replace alignment
# random replace compound
# random replace word

In [39]:
random.random()

0.5055752996165427

In [40]:
malaya.augmentation.socialmedia_form('Saya')

['sayak', 'saye']

In [41]:
malaya.augmentation.vowel_alternate('saya')

'sya'

In [42]:
d.check("Hello")

True

In [75]:
fast_text = malaya.language_detection.fasttext()
fast_text.predict([left[7106], right[7106], 'saya suka'])

['eng', 'malay', 'malay']

In [50]:
import copy

In [58]:
consonants = 'bcdfghjklmnpqrstvwxyz'

def augment(left, p_replace_shortword = 0.7, p_vowel_alternate = 0.8):
    
    left = _replace_compound(left, rules = copy.deepcopy(compound_normalizer))
    left = [(replace_shortword(word, rules = normalizer), False) if random.random() > p_replace_shortword else (word, True) for word in left.split()]
    left_ = []
    for l in left:
        if _is_number_regex(l[0].replace(',', '').replace('.', '')):
            left_.append(l[0])
            continue
        if l[0].isupper():
            left_.append(l[0])
            continue
        if l[0].istitle():
            left_.append(l[0])
            continue
        
        if l[1]:
            s = l[0]
                
            if random.random() > p_vowel_alternate:
                try:
                    s = malaya.augmentation.vowel_alternate(s)
                except:
                    pass
                try:
                    if random.random() and s[-1] == 'a' and s[-2] in consonants:
                        s = s[:-1] + 'e'
                except:
                    pass
        else:
            s = l[0]
        
        left_.append(case_of(l[0])(s))
    return ' '.join(left_)

In [83]:
malaya.augmentation.vowel_alternate('like')

'lke'

In [59]:
left[0]

'Most of them are teenagers or in their 20s.'

In [66]:
augment(left[0] + ', fuck my life')

'Most ofr them are teenagers or in their 20s., fml'

In [71]:
%%time

augment(left[0])

CPU times: user 305 µs, sys: 122 µs, total: 427 µs
Wall time: 431 µs


'Most of them are teenagers or in their 20s.'

In [76]:
from tqdm import tqdm
import traceback

def loop(rows):
    rows, _ = rows
    new_left, new_right, original = [], [], []
    for i in tqdm(range(len(rows))):
        left, right = rows[i][0], rows[i][1]
        if len(left.split()) > 100 or len(right.split()) > 100:
            continue
        langs = fast_text.predict([left, right])
        if langs[0] == 'malay':
            continue
        if langs[1] == 'eng':
            continue
        try:
            new_left_ = augment(left)
            if new_left_ != left:
                new_left.append(new_left_)
                new_right.append(right)
                original.append(left)
        except Exception as e:
            print(traceback.format_exc())
    return [[new_left, new_right]]

In [73]:
augment(left[1800000 + 3160])

"Yupp that's yoy you choud quit now"

In [84]:
r = loop((list(zip(left[1800000:1800010], right[1800000:1800010])),0))

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 1198.34it/s]


In [85]:
r

[[['Dhaa government has given us alots on*2 help but we need 2 get outte our minds fourth free assistence.',
   "Allah Almighty sez in the Qur'an Surat An-Nisa Verse 86: If you r honred wth honor then respond whit a better respect, or respond with duh sam.",
   'Apart fro tht said Baharudin Djafar, an Sulbar Police si also prepairing concrete measures to prvde security tp the public frm vairous possible acks of crime and terrorism over the years',
   'we ve graphed our parabolas.',
   'Mooe than 170 millon copys of his book have been sold with 40 languages worldwide, meny*2 of themmmmm into firms.',
   'SOE Effords tew Hellp Rescue Donggala and Rescue Earthquake and Tsunami Page all',
   'Of parents e chiden yoou bo not knw hwo are more usfull*2 to youuuuu.'],
  ['Kerajaan sudah memberikan banyak bantuan tetapi kita kena keluar daripada pemikiran untuk mendapat bantuan percuma.',
   "Allah SWT berfirman didalam Al-Qur'an Surat An-Nisa Ayat 86: Apabila kamu dihormati dengan suatu pengho

In [88]:
import mp

In [86]:
len(left)

3807616

In [89]:
r = mp.multiprocessing(list(zip(left[:1200000], right[:1200000])), loop, cores = 8)

100%|██████████████████████████████████████████████████████████████████████████████████████| 150000/150000 [02:23<00:00, 1043.73it/s]


In [90]:
en, ms = [], []
for i in range(len(r)):
    print(i, len(r[i][0]))
    en.extend(r[i][0])
    ms.extend(r[i][1])

0 123137
1 123110
2 123105
3 123253
4 123105
5 123171
6 123066
7 122843


In [91]:
len(ms), len(en)

(984790, 984790)

In [93]:
en[-10:]

["They promise to dowr sch outlying quarters ass Edinburgh's Westen and Granton harbours back the the heart of the existing sied.",
 'Not alot buts overdue is*2 when I do',
 "Ir 1967, Sultan Haji Omar 'Ali Saifuddien Sa'adul Khairi Waddien stepped down frm tehe throne and appointed his eldest son, Sultan Haji Hassanal Bolkiah Mu'izzaddin Waddaulah, tooo bacome the 29th Sultan of Brunei.",
 'It is understud that the investigation into the corruption of the government agentcy involvd nearly RM3 millon.',
 "Anwar is already a Memeber of Parliament and der country iz generally ready to accept him as Mahathir's replasments when tie time comes.",
 "The name given to Lakshamilavan's daughtr was Mom Chao Varnbimol Voravarn.",
 'The chemotherapy treatment for tuberculosis was only started in 1946 wiv teh dscovery of streptomycin und wos*5 followed by the intro of isniazid as the first effective antitubi drg in 1952.',
 'Dee Zip code used att First it 46192.',
 'You con compare thee*2 options th

In [92]:
ms[-10:]

["Mereka berjanji untuk menarik tempat terpencil seperti Edinburgh's Western dan Granton kembali ke tengah-tengah kota yang ada.",
 'Tidak banyak tetapi terlambat ketika saya melakukan',
 "Pada tahun 1967, Sultan Haji Omar 'Ali Saifuddien Sa'adul Khairi Waddien telah turun dari takhta dan melantik anakanda sulung baginda, Sultan Haji Hassanal Bolkiah Mu'izzaddin Waddaulah, menjadi Sultan Brunei ke-29.",
 'Difahamkan, siasatan kes rasuah agensi kerajaan tersebut melibatkan nilai hampir RM3 juta.',
 'Anwar pun sudah menjadi ahli Parlimen dan umumnya negara sentiasa bersedia menerimanya sebagai pengganti Mahathir apabila tiba masanya.',
 'Nama yang diberikan puteri Lakshamilavan adalah Mom Chao Varnbimol Voravarn.',
 'Rawatan kemoterapi bagi merawat tibi hanya mula ditemui pada 1946 dengan penemuan streptomisin dan diikuti pengenalan isoniazid sebagai ubat antitibi barisan pertama yang efektif pada 1952.',
 'Poskod yang digunakan di Kawitan adalah 46192.',
 'Anda boleh membandingkan tiga 

In [94]:
import json

with open('augmented-en-ms-v1.json', 'w') as fopen:
    json.dump({'en': en, 'ms': ms}, fopen)